In [10]:
import numpy as np
import math

class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.childs = []

# dataset
# data in columns
# row1: outlook, row2: temp, row3: humidity, row4: windy, row5: play golf
data= [['rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'sunny', 'rainy', 'overcast', 'overcast', 'sunny','sunny', 'overcast', 'rainy', 'overcast'],
       ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild','cool','mild','hot', 'cool'],
       ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high','high', 'normal','normal','high'],
       ['false', 'true', 'false', 'false', 'false', 'true', 'true', 'false', 'false', 'false', 'true', 'true', 'false', 'true','false','true','true','true'],
       ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes']]
data = np.asarray(data)

allAttributes = ['outlook', 'temp', 'humidity', 'windy']

# print(data.shape)

x = data[0:3,:] #attributes
y = data[4,:] #play golf

# calculate entropy
def calEntropy (p, n):
    #this function calculates the  entropy,means you have to implement the formula of entropy here.
    #write down your code here
    if p == 0 or n == 0: 
        return 0
 
    return -((p/(p+n)) * np.log2(p/(p+n))) - ((n/(p+n)) * np.log2(n/(p+n))) 

def entropy(dataset):
    p = len(dataset[-1,(dataset[-1,:]=='yes')]) #number of positive labels
    n = len(dataset[-1,(dataset[-1,:]=='no')]) #number of positive labels
    return calEntropy(p,n)

# attr is row index of specific attribute in dataset array
def entropyAfter(dataset, attr):
    uniqueValues, count = np.unique(dataset[attr, :], return_counts=True)
    i=0
    ent = 0
    for val in uniqueValues:
        p = len(dataset[attr, ( (dataset[attr,:]==val) & (dataset[-1,:]=='yes') )])
        n = len(dataset[attr, ( (dataset[attr,:]==val) & (dataset[-1,:]=='no') )])
        ent  = ent + ( count[i]/len(dataset[attr,:]) )*calEntropy(p,n)
        i = i+1
    return ent

# print(entropyAfter(data, 0))#testing entropy after function

# attributes = list of attributes currently in dataset (narrows down as tree grows)

def infoGain(dataset, attributes, attribute):
    #implement the formula of info gain here
    #you have to write your own piece of code to compute info gain
    ind = attributes.index(attribute)
    return entropy(dataset) - entropyAfter(dataset, ind)

# print(infoGain(data, allAttributes, "outlook")) #testing infoGain function

def getNode(dataset, attributes):
    maxInfo=-1
    maxAttr = ''
    ans = entropy(dataset)
    for attr in attributes:
        info = infoGain(dataset, attributes, attr)
        if info>maxInfo:
            maxInfo = info
            maxAttr = attr
    return maxAttr, maxInfo, ans

# print("\nRoot decision: ", getNode(data, allAttributes), "\n") #test getNode function

def buildTree(dataset, attributes):
    attr, info, ans = getNode(dataset, attributes)
    root = Node()
    if ans == 1:   
        #print("always yess!!!\n")
        root.decision = 'yes'
        #root.value = "arrived at yes"
        return root
    if ans == 0:
        #print("always no!!!\n")
        root.decision = dataset[-1,1]
        #root.value = "arrived at no"
        return root

    root.decision = attr

    attrRow = attributes.index(attr) #find row correspond to the attribute attr
    possibleValues = np.unique(dataset[attrRow, :]) # unique values of attribute
    for val in possibleValues:
        tempData = dataset[:, (dataset[attrRow, :]==val)] # narrowing data
        tempData = np.delete(tempData, attrRow, 0) # removing attribute row
        tempAttrList = attributes[:]
        tempAttrList.remove(attr) # remove attribute from list of attributes
        tempTree = buildTree(tempData, tempAttrList) # recursively build tree
        tempTree.value = val
        root.childs.append(tempTree)

    return root

def traverse(root):
    print(root.value)
    print(root.decision)
    
    print("")

    for child in root.childs:
        traverse(child)

print("Decision Tree\n")
root = buildTree(data, allAttributes)
root.value = 'Start'
traverse(root)

#predict using decision tree
def predict(node, attributeHeadings, attributeValues):
    #you have to think about and write your own piece of code to make the above constructed decision tree predict results
    #you have to write your own piece of code here
    
    if node.childs == []:
        return node.decision
    
    for nextnode in node.childs:
        if nextnode.value in attributeValues:
            ans = predict(nextnode, attributeHeadings, attributeValues)
        if nextnode.value in attributeHeadings:
            ans = predict(nextnode, attributeHeadings, attributeValues)
    
    return ans

Decision Tree

Start
outlook

overcast
yes

rainy
yes

sunny
windy

false
yes

true
no



In [2]:
testcase1 = predict(root, allAttributes, ["sunny", "cool", "high", "false"])
print(testcase1)

yes


In [3]:
testcase2 = predict(root, allAttributes, ["overcast", "mild", "normal", "true"])
print(testcase2)

yes


In [4]:
testcase3 = predict(root, allAttributes, ["rainy", "hot", "normal", "true"])
print(testcase3)

yes


In [5]:
testcase4 = predict(root, allAttributes, ["overcast", "cool", "high", "true"])
print(testcase4)

yes


In [11]:
specialtest = predict(root, allAttributes, ["sunny", "cool", "high", "false"])
print(specialtest)

yes
